In [1]:
%matplotlib inline 
import numpy as np 
import scipy as sp 
import matplotlib as mpl
import matplotlib.cm as cm 
import matplotlib.pyplot as plt
import pandas as pd 
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set(style="whitegrid")
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("data.csv")
data = data.reset_index(drop=True)

In [3]:
result = pd.read_excel("Result.xlsx")
result = result.reset_index(drop=True)

In [4]:
data.head()

,match_event_id,shot_id_number,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,team_name,date_of_game,home/away,type_of_shot,type_of_combined_shot,match_id,team_id
0,10.0,1,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,NaN,Right Side(R),Mid Range,16-24 ft.,Manchester United,31-10-00,MANU @ POR,shot - 30,Shot-None,20000012,1610612747
1,12.0,2,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,Left Side(L),Mid Range,8-16 ft.,Manchester United,31-10-00,MANU @ POR,shot - 45,Shot-None,20000012,1610612747
2,35.0,3,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,Left Side Center(LC),Mid Range,16-24 ft.,Manchester United,31-10-00,NaN,shot - 25,Shot-None,20000012,1610612747
3,43.0,4,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,Right Side Center(RC),Mid Range,16-24 ft.,Manchester United,31-10-00,MANU @ POR,Shot-None,shot - 3,20000012,1610612747
4,155.0,5,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,Center(C),Goal Area,Less Than 8 ft.,NaN,31-10-00,MANU @ POR,Shot-None,shot - 1,20000012,1610612747


In [5]:
result.head()

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,home/away,type_of_shot,type_of_combined_shot,match_id,team_id,time_remaining,year,month,Last
0,10.0,1,167.0,72.0,1.0,0.0,38.0,NaN,Right Side(R),Mid Range,16-24 ft.,MANU @ POR,shot - 30,Shot-None,20000012,1610612747,10.450000,2000.0,10.0,1.0
1,254.0,8,1.0,28.0,3.0,0.0,22.0,NaN,Center(C),Goal Area,Less Than 8 ft.,MANU @ POR,Shot-None,shot - 3,20000012,1610612747,8.083333,2000.0,10.0,1.0
2,100.0,17,0.0,0.0,1.0,0.0,20.0,NaN,NaN,Goal Area,Less Than 8 ft.,MANU vs. UTA,Shot-None,shot - 4,20000019,1610612747,0.016667,2000.0,1.0,1.0
3,249.0,20,0.0,0.0,3.0,0.0,20.0,NaN,Center(C),NaN,Less Than 8 ft.,MANU vs. UTA,shot - 4,Shot-None,20000019,1610612747,10.766667,2000.0,1.0,1.0
4,265.0,22,134.0,127.0,3.0,0.0,38.0,NaN,Right Side Center(RC),Mid Range,16-24 ft.,MANU vs. UTA,shot - 24,Shot-None,20000019,1610612747,9.066667,2000.0,1.0,NaN


In [6]:
result = result.dropna(subset=['Last'])
data = data.dropna(subset=['is_goal'])

In [7]:
result = result.drop(columns='Last')
result = result.drop(columns='is_goal')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24429 entries, 1 to 30696
Data columns (total 21 columns):
match_event_id           23189 non-null float64
shot_id_number           24429 non-null int64
location_x               23273 non-null float64
location_y               23184 non-null float64
remaining_min            23185 non-null float64
power_of_shot            23229 non-null float64
knockout_match           23217 non-null float64
game_season              19797 non-null object
remaining_sec            23179 non-null float64
distance_of_shot         23172 non-null float64
is_goal                  24429 non-null float64
area_of_shot             23247 non-null object
shot_basics              23146 non-null object
range_of_shot            23181 non-null object
team_name                23217 non-null object
date_of_game             23192 non-null object
home/away                23215 non-null object
type_of_shot             24429 non-null object
type_of_combined_shot    24429 non-nu

In [9]:
data['time_remaining'] = data.remaining_min + data.remaining_sec/60
data = data.drop(columns=['remaining_min','remaining_sec','game_season'])


In [10]:
data.date_of_game = pd.to_datetime(data.date_of_game)

In [11]:
data['year'] = data['date_of_game'].dt.year
data['month'] = data['date_of_game'].dt.month
data = data.drop(columns='date_of_game')


In [12]:
data.knockout_match = data.knockout_match.astype('object')
data.year = data.year.astype('object')
data.month = data.month.astype('object')

result.knockout_match = result.knockout_match.astype('object')
result.year = result.year.astype('object')
result.month = result.month.astype('object')

In [13]:
data.describe()

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,distance_of_shot,is_goal,match_id,team_id,time_remaining
count,23189.000000,24429.000000,23273.000000,23184.000000,23229.000000,23172.000000,24429.000000,2.442900e+04,2.442900e+04,21995.000000
mean,249.575402,15329.854804,7.494178,91.461180,2.521503,33.467159,0.445331,2.474317e+07,1.610613e+09,5.351919
std,150.052347,8860.893604,110.126457,88.041628,1.153056,9.380114,0.497013,7.745041e+06,0.000000e+00,3.474208
min,2.000000,2.000000,-250.000000,-44.000000,1.000000,20.000000,0.000000,2.000001e+07,1.610613e+09,0.000000
25%,111.000000,7654.000000,-67.000000,4.000000,1.000000,25.000000,0.000000,2.050006e+07,1.610613e+09,2.350000
50%,254.000000,15340.000000,0.000000,74.000000,3.000000,35.000000,0.000000,2.090034e+07,1.610613e+09,5.050000
75%,369.000000,22969.000000,95.000000,160.000000,3.000000,41.000000,1.000000,2.960027e+07,1.610613e+09,8.316667
max,653.000000,30697.000000,248.000000,791.000000,7.000000,99.000000,1.000000,4.990009e+07,1.610613e+09,11.900000


In [14]:
data = data.drop(columns='team_name')

In [15]:
categorical_variables = data.dtypes.loc[data.dtypes=='object'].index

In [16]:
categorical_variables

Index(['knockout_match', 'area_of_shot', 'shot_basics', 'range_of_shot', 'home/away', 'type_of_shot', 'type_of_combined_shot', 'year', 'month'], dtype='object')

In [17]:
continuous_varaible = [x for x in data.columns if x not in categorical_variables]

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24429 entries, 1 to 30696
Data columns (total 19 columns):
match_event_id           23189 non-null float64
shot_id_number           24429 non-null int64
location_x               23273 non-null float64
location_y               23184 non-null float64
power_of_shot            23229 non-null float64
knockout_match           23217 non-null object
distance_of_shot         23172 non-null float64
is_goal                  24429 non-null float64
area_of_shot             23247 non-null object
shot_basics              23146 non-null object
range_of_shot            23181 non-null object
home/away                23215 non-null object
type_of_shot             24429 non-null object
type_of_combined_shot    24429 non-null object
match_id                 24429 non-null int64
team_id                  24429 non-null int64
time_remaining           21995 non-null float64
year                     23192 non-null object
month                    23192 non-null o

In [19]:
data.head()

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,home/away,type_of_shot,type_of_combined_shot,match_id,team_id,time_remaining,year,month
1,12.0,2,-157.0,0.0,1.0,0,35.0,0.0,Left Side(L),Mid Range,8-16 ft.,MANU @ POR,shot - 45,Shot-None,20000012,1610612747,10.366667,2000,10
2,35.0,3,-101.0,135.0,1.0,0,36.0,1.0,Left Side Center(LC),Mid Range,16-24 ft.,NaN,shot - 25,Shot-None,20000012,1610612747,7.750000,2000,10
3,43.0,4,138.0,175.0,1.0,0,42.0,0.0,Right Side Center(RC),Mid Range,16-24 ft.,MANU @ POR,Shot-None,shot - 3,20000012,1610612747,6.866667,2000,10
4,155.0,5,0.0,0.0,2.0,0,20.0,1.0,Center(C),Goal Area,Less Than 8 ft.,MANU @ POR,Shot-None,shot - 1,20000012,1610612747,NaN,2000,10
5,244.0,6,-145.0,-11.0,3.0,0,34.0,0.0,Left Side(L),Mid Range,8-16 ft.,MANU @ POR,shot - 17,Shot-None,20000012,1610612747,9.533333,2000,10


In [20]:
data = data.reset_index(drop=True)
result = result.reset_index(drop=True)

## Missing Data

In [21]:
# Detecting missing values
data.isnull().sum()

match_event_id           1240
shot_id_number              0
location_x               1156
location_y               1245
power_of_shot            1200
knockout_match           1212
distance_of_shot         1257
is_goal                     0
area_of_shot             1182
shot_basics              1283
range_of_shot            1248
home/away                1214
type_of_shot                0
type_of_combined_shot       0
match_id                    0
team_id                     0
time_remaining           2434
year                     1237
month                    1237
dtype: int64

In [22]:
for i in categorical_variables:
    data[str(i)].fillna(data[str(i)].mode()[0], inplace=True)

data = data.fillna(data.mean())


In [23]:
for i in categorical_variables:
    result[str(i)].fillna(result[str(i)].mode()[0], inplace=True)

result = result.fillna(data.mean())

In [24]:
# data = data.dropna(axis = 0, how = 'any')
# data = data.reset_index(drop=True)

In [25]:
data.isnull().sum()

match_event_id           0
shot_id_number           0
location_x               0
location_y               0
power_of_shot            0
knockout_match           0
distance_of_shot         0
is_goal                  0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
type_of_shot             0
type_of_combined_shot    0
match_id                 0
team_id                  0
time_remaining           0
year                     0
month                    0
dtype: int64

In [26]:
result.isnull().sum()

match_event_id           0
shot_id_number           0
location_x               0
location_y               0
power_of_shot            0
knockout_match           0
distance_of_shot         0
area_of_shot             0
shot_basics              0
range_of_shot            0
home/away                0
type_of_shot             0
type_of_combined_shot    0
match_id                 0
team_id                  0
time_remaining           0
year                     0
month                    0
dtype: int64

## -----------------------------------------------------------

In [27]:
data[categorical_variables].apply(lambda x: len(x.unique()))

knockout_match            2
area_of_shot              6
shot_basics               7
range_of_shot             5
home/away                74
type_of_shot             58
type_of_combined_shot     7
year                     21
month                    12
dtype: int64

In [28]:
for var in categorical_variables:
    print(var)
    print(data[var].value_counts())
    print('\n')

knockout_match
0.0    21046
1.0     3383
Name: knockout_match, dtype: int64


area_of_shot
Center(C)                11359
Right Side Center(RC)     3616
Right Side(R)             3504
Left Side Center(LC)      3031
Left Side(L)              2848
Mid Ground(MG)              71
Name: area_of_shot, dtype: int64


shot_basics
Mid Range          10771
Goal Area           5323
Penalty Spot        4245
Goal Line           3515
Right Corner         302
Left Corner          216
Mid Ground Line       57
Name: shot_basics, dtype: int64


range_of_shot
Less Than 8 ft.    8312
16-24 ft.          6220
8-16 ft.           5076
24+ ft.            4751
Back Court Shot      70
Name: range_of_shot, dtype: int64


home/away
MANU @ SAS      1981
MANU @ PHX       701
MANU vs. SAS     693
MANU @ SAC       686
MANU vs. PHX     650
MANU vs. HOU     642
MANU @ DEN       639
MANU @ HOU       615
MANU @ POR       614
MANU vs. DEN     581
MANU vs. SAC     579
MANU vs. POR     559
MANU @ MIN       554
MANU vs. MIN  

In [29]:
# data.plot('location_x','location_y',c= 'is_goal',colormap='autumn',kind='scatter')
# plt.show()

In [30]:
# goals=data[data["is_goal"]==1]

In [31]:
# len(goals)

In [32]:
# fig=plt.figure(figsize=(8,6))
# goals.year.value_counts().sort_index().plot()
# plt.xlabel("Time")
# plt.ylabel("Number of goals")
# plt.title("Number of goals against Time during match")
# plt.show()

In [33]:
# fig=plt.figure(figsize=(8,6))
# plt.hist(goals[goals["side"]==1]["time"],width=1,bins=100,color="cyan",label="home goals")   
# plt.hist(goals[goals["side"]==2]["time"],width=1,bins=100,color="grey",label="away goals") 
# plt.xlabel("Minutes")
# plt.ylabel("Number of goals")
# plt.legend()
# plt.title("Number of goals (by home/away side) against Time during match")

In [34]:
data.columns

Index(['match_event_id', 'shot_id_number', 'location_x', 'location_y', 'power_of_shot', 'knockout_match', 'distance_of_shot', 'is_goal', 'area_of_shot', 'shot_basics', 'range_of_shot', 'home/away', 'type_of_shot', 'type_of_combined_shot', 'match_id', 'team_id', 'time_remaining', 'year', 'month'], dtype='object')

In [35]:
dependent_variable = 'is_goal'
independent_variable = [x for x in data.columns if x not in [dependent_variable]]

In [36]:
independent_variable

['match_event_id',
 'shot_id_number',
 'location_x',
 'location_y',
 'power_of_shot',
 'knockout_match',
 'distance_of_shot',
 'area_of_shot',
 'shot_basics',
 'range_of_shot',
 'home/away',
 'type_of_shot',
 'type_of_combined_shot',
 'match_id',
 'team_id',
 'time_remaining',
 'year',
 'month']

In [37]:
data.columns

Index(['match_event_id', 'shot_id_number', 'location_x', 'location_y', 'power_of_shot', 'knockout_match', 'distance_of_shot', 'is_goal', 'area_of_shot', 'shot_basics', 'range_of_shot', 'home/away', 'type_of_shot', 'type_of_combined_shot', 'match_id', 'team_id', 'time_remaining', 'year', 'month'], dtype='object')

In [38]:
categorical_variables

Index(['knockout_match', 'area_of_shot', 'shot_basics', 'range_of_shot', 'home/away', 'type_of_shot', 'type_of_combined_shot', 'year', 'month'], dtype='object')

In [39]:
X = data[independent_variable]

In [40]:
X.tail()

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,area_of_shot,shot_basics,range_of_shot,home/away,type_of_shot,type_of_combined_shot,match_id,team_id,time_remaining,year,month
24424,382.0,30692,0.0,0.00000,4.0,1.0,20.0,Center(C),Goal Area,Less Than 8 ft.,MANU vs. IND,shot - 4,Shot-None,49900088,1610612747,5.351919,2009.0,12.0
24425,397.0,30693,1.0,48.00000,4.0,1.0,24.0,Center(C),Mid Range,Less Than 8 ft.,MANU vs. IND,shot - 1,Shot-None,49900088,1610612747,6.083333,2000.0,6.0
24426,426.0,30695,-134.0,166.00000,4.0,1.0,41.0,Left Side Center(LC),Mid Range,16-24 ft.,MANU vs. IND,Shot-None,shot - 3,49900088,1610612747,3.466667,2000.0,6.0
24427,448.0,30696,31.0,267.00000,4.0,1.0,46.0,Center(C),Penalty Spot,Less Than 8 ft.,MANU vs. IND,shot - 26,Shot-None,49900088,1610612747,2.166667,2000.0,6.0
24428,471.0,30697,1.0,91.46118,4.0,1.0,27.0,Center(C),Goal Line,Less Than 8 ft.,MANU vs. IND,shot - 45,Shot-None,49900088,1610612747,0.650000,2000.0,6.0


In [41]:
# X.iloc[:24429, :]

In [42]:
M = data[independent_variable].append(result[independent_variable])

In [43]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for var in categorical_variables:
    M[var] = le.fit_transform(M[var])

In [44]:
data_ = M.iloc[:24429, :]

In [45]:
data_['is_goal'] = data['is_goal']

In [46]:
result = M.iloc[24429:, :]

In [47]:
result

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,area_of_shot,shot_basics,range_of_shot,home/away,type_of_shot,type_of_combined_shot,match_id,team_id,time_remaining,year,month
0,10.000000,1,167.000000,72.00000,1.000000,0,38.000000,5,4,0,28,25,0,20000012,1610612747,10.450000,4,9
1,254.000000,8,1.000000,28.00000,3.000000,0,22.000000,0,0,4,28,0,4,20000012,1610612747,8.083333,4,9
2,100.000000,17,0.000000,0.00000,1.000000,0,20.000000,0,0,4,71,0,5,20000019,1610612747,0.016667,4,0
3,249.000000,20,0.000000,0.00000,3.000000,0,20.000000,0,4,4,71,35,0,20000019,1610612747,10.766667,4,0
4,4.000000,33,163.000000,76.00000,1.000000,0,37.000000,5,4,0,35,29,0,20000047,1610612747,11.433333,4,3
5,8.000000,34,70.000000,194.00000,1.000000,0,40.000000,4,4,0,35,0,4,20000047,1610612747,10.966667,4,3
6,26.000000,35,1.000000,19.00000,1.000000,0,21.000000,0,0,4,35,34,0,20000047,1610612747,7.550000,4,3
7,37.000000,36,-12.000000,91.46118,1.000000,0,21.000000,0,0,4,35,34,0,20000047,1610612747,5.966667,4,3
8,53.000000,37,1.000000,4.00000,1.000000,0,20.000000,0,0,4,35,0,5,20000047,1610612747,4.150000,4,3
9,165.000000,38,-117.000000,116.00000,2.000000,0,36.000000,1,4,4,35,2,0,20000047,1610612747,5.550000,13,11


In [48]:
data = data_

## Splitting our data into training and testing data

In [49]:
# from sklearn.model_selection import train_test_split

In [50]:
# train , test = train_test_split(data, test_size=.2)

In [51]:
# train.head()

In [52]:
# print(len(data))
# print(len(train))
# print(len(test))

In [53]:
from sklearn.ensemble import  RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier

In [54]:
model = XGBClassifier()
model.fit(data[independent_variable],data[dependent_variable])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [55]:
prediction = model.predict(result[independent_variable])

In [56]:
prediction

array([0., 0., 1., ..., 1., 0., 0.])

In [57]:
# from sklearn.metrics import accuracy_score
# accuracy_score(prediction,data[dependent_variable])

In [58]:
# from sklearn.metrics import mean_absolute_error
# MAE = mean_absolute_error(prediction,data[dependent_variable])

In [59]:
# Score = 1/(1+ MAE)

In [60]:
# Score

In [61]:
result.head()

,match_event_id,shot_id_number,location_x,location_y,power_of_shot,knockout_match,distance_of_shot,area_of_shot,shot_basics,range_of_shot,home/away,type_of_shot,type_of_combined_shot,match_id,team_id,time_remaining,year,month
0,10.0,1,167.0,72.0,1.0,0,38.0,5,4,0,28,25,0,20000012,1610612747,10.450000,4,9
1,254.0,8,1.0,28.0,3.0,0,22.0,0,0,4,28,0,4,20000012,1610612747,8.083333,4,9
2,100.0,17,0.0,0.0,1.0,0,20.0,0,0,4,71,0,5,20000019,1610612747,0.016667,4,0
3,249.0,20,0.0,0.0,3.0,0,20.0,0,4,4,71,35,0,20000019,1610612747,10.766667,4,0
4,4.0,33,163.0,76.0,1.0,0,37.0,5,4,0,35,29,0,20000047,1610612747,11.433333,4,3


In [62]:
Output = pd.DataFrame()

In [63]:
Output['shot_id_number'] = result['shot_id_number']

In [64]:
Output

,shot_id_number
0,1
1,8
2,17
3,20
4,33
5,34
6,35
7,36
8,37
9,38


In [65]:
Output['is_goal'] = prediction

In [69]:
Output.to_csv('Output.csv',index=False)